# Scraping the espncricinfo website using selenium

In [34]:
import time 
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys 
import pandas as pd

# Add for loop to change page number 
start_url = 'http://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;orderby=start;page=154;template=results;type=team;view=results'
num_matches_per_page = 4
df = pd.DataFrame(columns = ['Date','Venue','Team_1','Team_2','Innings_no','Overs','Ball_Event','Comment'])
go_to_commentary(start_url,num_matches_per_page,df)



Checking for mac64 chromedriver:75.0.3770.8 in cache
Driver found in /Users/murali/.wdm/chromedriver/75.0.3770.8/mac64/chromedriver
Result
<class 'str'>

Checking for mac64 chromedriver:75.0.3770.8 in cache
Driver found in /Users/murali/.wdm/chromedriver/75.0.3770.8/mac64/chromedriver
251
305
           Date                                Venue   Team_1    Team_2  \
0   Nov 20 2015  Dubai International Cricket Stadium  England  Pakistan   
1   Nov 20 2015  Dubai International Cricket Stadium  England  Pakistan   
2   Nov 20 2015  Dubai International Cricket Stadium  England  Pakistan   
3   Nov 20 2015  Dubai International Cricket Stadium  England  Pakistan   
4   Nov 20 2015  Dubai International Cricket Stadium  England  Pakistan   

  Innings_no Overs Ball_Event  \
0          2  40.4          W   
1          2  40.3          0   
2          2  40.2          0   
3          2  40.1          0   
4          2  39.6          0   

                                             Comment  


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=75.0.3770.90)


In [33]:

# team_abbrev = {'South Africa': 'SA','Australia':'AUS','Pakistan':'PAK','West Indies':'WI','Sri Lanka':'SL',
#                 'England':'ENG','Bangladesh':'BDESH','New Zealand':'NZ','Afghanistan':'AFG','India':'INDIA'}

def go_to_commentary(start_url,num,df):
    '''
    function to automate mouse operations that lead from start_url to the commentary for all matches on that webpage
    
    '''
    # Iterating over alternate rows present on that webpage (matches are repeated)
    for i in range(1,num,2):
        driver = webdriver.Chrome(ChromeDriverManager().install())
        act = ActionChains(driver)
        driver.get(start_url)
        
        block = driver.find_element_by_xpath("//*[@id='ciHomeContentlhs']/div[3]/table[3]")
        elements = block.find_elements_by_tag_name('img') # finds all images in the specified table
        # NOTE: first image is the black up arrow, use from the second element to get all the hidden menu image icons
        
        act.move_to_element(elements[i]).perform()
        scorecard = driver.find_element_by_link_text('Match scorecard')
        act.click(scorecard).perform()         # clicks match scorecard link 
        elt = driver.find_element_by_class_name("cscore_link--button")
        status = elt.find_element_by_class_name("cscore_time")
        print(status.text)
        print(type(status.text))
        # Leaving out abandoned matches 
        if status.text=="Result":
            stadium = driver.find_element_by_xpath("//*[@id='main-container']/div/div[3]/div[1]/div[6]/article/div/div/div[1]/h4/a/span")
            stadium = stadium.text             # getting venue details  
            elt = driver.find_element_by_class_name("cscore_link--button")
            date = elt.find_element_by_class_name("cscore_info-overview")
            date_text = date.text.split(',')[2]  # getting date 
            commentary = driver.find_element_by_link_text('Commentary')
            act.click(commentary).perform()        # clicks commentary link
            scrape_data(driver.current_url,stadium,date_text,df)        # function call to scrape_data() for every match 
            driver.quit()

        

def scrape_data(url,stadium,date,df):
    '''
    function to visit the url passed and scrape the commentary for every ball for both the innings

    '''    
    driver2 = webdriver.Chrome(ChromeDriverManager().install())
    driver2.get(url)
    # Obtaining team names 
    teams = driver2.find_elements_by_class_name('cscore_name--long')
    team_names = []
    for team in teams:
        if len(team.text)!=0:
            team_names.append(team.text)
    time.sleep(2)  

    #scrolling till we reach the end of the page
    driver2.maximize_window();
    elm = driver2.find_element_by_tag_name('html')
    for i in range(12):
        elm.send_keys(Keys.END)
        time.sleep(3)
        
    # Code to extract ball-by-ball status and commentary (first for innings 2)  
    innings = 2 
    wrappers = driver2.find_elements_by_class_name('item-wrapper')
    print(len(wrappers))
    for wrapper in wrappers:
        over = wrapper.find_element_by_class_name('time-stamp')
        over_value = over.text
        ball_score = wrapper.find_element_by_class_name('over-score')
        ball_event = ball_score.text
        comment = wrapper.find_element_by_class_name('description')
        comment_made = comment.text
        innings_data = [date,stadium,team_names[0],team_names[1],innings,over_value,ball_event,comment_made]
        row = pd.Series(dict(zip(df.columns,innings_data)))
        df = df.append(row,ignore_index = True)
    
    new_url = driver2.current_url + '?innings=1'
    driver2.get(new_url)
    driver2.maximize_window();
    
    #scrolling till we reach the end of the page
    elm = driver2.find_element_by_tag_name('html')
    for i in range(12):
        elm.send_keys(Keys.END)
        time.sleep(3)
        
    # Code to extract ball-by-ball status and commentary (now for innings 1)   
    innings = 1 
    wrappers = driver2.find_elements_by_class_name('item-wrapper')
    print(len(wrappers))
    for wrapper in wrappers:
        over = wrapper.find_element_by_class_name('time-stamp')
        over_value = over.text
        ball_score = wrapper.find_element_by_class_name('over-score')
        ball_event = ball_score.text
        comment = wrapper.find_element_by_class_name('description')
        comment_made = comment.text
        innings_data = [date,stadium,team_names[0],team_names[1],innings,over_value,ball_event,comment_made]
        row = pd.Series(dict(zip(df.columns,innings_data)))
        df = df.append(row,ignore_index = True)
    driver2.quit()
    print(df.head())



In [50]:
import os
print(os.getcwd())

/Users/murali


In [4]:

import time
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
#start_url = 'http://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;orderby=start;page=154;template=results;type=team;view=results'
#chr_opt = webdriver.ChromeOptions()
#chr_opt.add_extension(os.getcwd()+'/adblocker.crx')
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.google.com")
time.sleep(5)
driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND+"w")




Checking for mac64 chromedriver:75.0.3770.90 in cache
Driver found in /Users/murali/.wdm/chromedriver/75.0.3770.90/mac64/chromedriver
